<a href="https://colab.research.google.com/github/vihan-lakshman/mutagenic/blob/main/old_code_do_not_use/masking_pipeline_SH2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation Dataset

In [ ]:
!pip install Bio

In [ ]:
import torch
import pandas as pd
import os
import random
from sklearn.metrics.pairwise import cosine_similarity
#df = pd.read_csv('/content/prelim-deletion-validation-dataset-functional-annotations-with-interpro.csv')

membrane_terms = "IPR000980" #set as empty string if you want to submit GO ID
num_residues_to_mask = 100

In [ ]:
if os.path.exists("/content/merged_data_noduplicates_new.csv"):
  merged_df = pd.read_csv('/content/merged_data_noduplicates_new.csv')
else:
  esm3_test_df = pd.read_csv('/content/selectedrcsbpdb.tsv', sep='\t', header = None)

  # Drop the last two columns
  esm3_test_df = esm3_test_df.iloc[:, :-2]
  esm3_test_df.columns = ["Protein Accession", "Seq Digest", "Seq Length", "Analysis", "Signature Accession", "Signature description", "Start location", "Stop location", "Score", "Status", "Date", "InterPro accession", "InterPro description"]

  merged_df = esm3_test_df.groupby('Protein Accession').agg({
      'Seq Length': 'first',
      'InterPro accession': lambda x: ';'.join(x[x != '-'].unique()),
      'InterPro description': lambda x: ';'.join(x[x != '-'].unique())
  }).reset_index()

  merged_df = merged_df.rename(columns={"InterPro accession": "InterPro_original", "InterPro description": "InterPro_original_description"})


interpro_go_df = pd.read_csv('/content/InterProDescriptions.tsv', sep='\t')

# Initialize an empty string to store the accessions
if membrane_terms != "":
  list_of_membrane_terms = membrane_terms
else:
  list_of_membrane_terms = ""

  # Iterate through the DataFrame rows
  for index, row in interpro_go_df.iterrows():
      go_terms = row['GO Terms']
      if isinstance(go_terms, str) and "GO:0016020" in go_terms:  # Check if the value is a string and contains "GO:0016020"
          accession = row['Accession']
          if isinstance(accession, str):  # Check if the accession is a string
            if list_of_membrane_terms == "":
              list_of_membrane_terms = accession
            else:
              list_of_membrane_terms += ';' + accession

# Add the new column
merged_df['InterPro_target'] = [list_of_membrane_terms] * len(merged_df)
df = merged_df
df

,Protein Accession,Seq Length,InterPro_original,InterPro_original_description,InterPro_target
0,5R1U_1|Chain,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000980
1,6KBQ_1|Chain,373,NaN,NaN,IPR000980
2,6KFD_1|Chain,162,IPR035992,Ricin B-like lectins,IPR000980
3,6L9T_1|Chain,595,IPR019791;IPR037120;IPR010255,"Haem peroxidase, animal-type;Haem peroxidase d...",IPR000980
4,6LM2_1|Chain,184,IPR012674;IPR015304,Calycin;ZinT domain,IPR000980
...,...,...,...,...,...
60,8WDG_1|Chain,385,IPR001998;IPR036237;IPR013022;IPR013453,Xylose isomerase;Xylose isomerase-like superfa...,IPR000980
61,8YJJ_1|Chain,190,IPR001137;IPR013319;IPR013320;IPR018208;IPR033...,Glycoside hydrolase family 11;Glycoside hydrol...,IPR000980
62,9C6E_1|Chain,365,IPR014756;IPR000698;IPR011022;IPR017864;IPR014...,Immunoglobulin E-set;Arrestin;Arrestin C-termi...,IPR000980
63,9EAT_1|Chain,220,IPR015892;IPR001765;IPR036874,"Carbonic anhydrase, prokaryotic-like, conserve...",IPR000980


In [ ]:
from Bio import SeqIO
# Read FASTA file into a dictionary
def parse_fasta(fasta_file):
    fasta_dict = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        accession = record.id.split("|")[0]  # Extract only the part before the first "|"
        fasta_dict[accession] = str(record.seq)
    return fasta_dict

# Update DataFrame with sequences
def update_dataframe_with_sequences(df, fasta_dict):
    # Preprocess 'Protein Accession' column
    df["Protein Accession"] = df["Protein Accession"].str.split("|").str[0]
    df["Sequence"] = df["Protein Accession"].map(fasta_dict)
    return df

# Example usage
fasta_file = "/content/selectedrcsbpdb.fasta"
fasta_dict = parse_fasta(fasta_file)
df = update_dataframe_with_sequences(df, fasta_dict)

df

,Protein Accession,Seq Length,InterPro_original,InterPro_original_description,InterPro_target,Sequence
0,5R1U_1,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000980,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
1,6KBQ_1,373,NaN,NaN,IPR000980,MVLLVIGLPLVSLVVALVAAAAPDSQVCDVDSTATCKITATPSQFQ...
2,6KFD_1,162,IPR035992,Ricin B-like lectins,IPR000980,LTCDQLPKAAINPIQEFIDSNPLEFEYVLTETFECTTRIYVQPARW...
3,6L9T_1,595,IPR019791;IPR037120;IPR010255,"Haem peroxidase, animal-type;Haem peroxidase d...",IPR000980,SWEVGCGAPVPLVKCDENSPYRTITGDCNNRRSPALGAANRALARW...
4,6LM2_1,184,IPR012674;IPR015304,Calycin;ZinT domain,IPR000980,ALTEVEQKAANGVFDDANVQNRTLSDWDGVWQSVYPLLQSGKLDPV...
...,...,...,...,...,...,...
60,8WDG_1,385,IPR001998;IPR036237;IPR013022;IPR013453,Xylose isomerase;Xylose isomerase-like superfa...,IPR000980,YQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVRRLAELGA...
61,8YJJ_1,190,IPR001137;IPR013319;IPR013320;IPR018208;IPR033...,Glycoside hydrolase family 11;Glycoside hydrol...,IPR000980,QTIQPGTGYNNGYFYSYWNDGHGGVTYTNGPGGQFSVNWSNSGNFV...
62,9C6E_1,365,IPR014756;IPR000698;IPR011022;IPR017864;IPR014...,Immunoglobulin E-set;Arrestin;Arrestin C-termi...,IPR000980,ANKPAPNHVIFKKISRDKSVTIYLGKRDYIDHVERVEPVDGVVLVD...
63,9EAT_1,220,IPR015892;IPR001765;IPR036874,"Carbonic anhydrase, prokaryotic-like, conserve...",IPR000980,MKDIDTLISNNALWSKMLVEEDPGFFEKLAQAQKPRFLWIGCSDSR...


In [ ]:
# Dictionary to hold the results
embeddings_dict = {}

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    entry = row['Protein Accession']
    interpro_target = row['InterPro_target']
    interpro_original = row['InterPro_original']

    # Skip rows where 'Interpro' is None
    if pd.isna(interpro_target) or not interpro_target.strip() or pd.isna(interpro_original) or not interpro_target.strip():
        continue

    # Split the InterPro IDs by semicolons
    interpro_ids_target = interpro_target.split(';')
    # interpro_ids_target = interpro_ids_target[:-1]
    interpro_ids_original = interpro_original.split(';')
    # interpro_ids_original = interpro_ids_original[:-1]

    # Initialize entry in the dictionary if not present
    if entry not in embeddings_dict:
        embeddings_dict[entry] = {
            'InterPro_ids_target': interpro_ids_target,
            'InterPro_ids_original': interpro_ids_original
        }


In [ ]:
first_key, first_value = next(iter(embeddings_dict.items()))
print(first_key, first_value['InterPro_ids_target'])

5R1U_1 ['IPR000980']


In [ ]:
pip install esm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login()

In [ ]:
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to("cuda")

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

data/esm3_entry.list:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

data/entry_list_safety_29026.list:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

...

esm3_sm_open_v1.pth:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

esm3_structure_encoder_v0.pth:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

data/tag_dict_4_safety_filtered.json:   0%|          | 0.00/569k [00:00<?, ?B/s]

esm3_structure_decoder_v0.pth:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/esm/pretrained.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [ ]:
import torch.nn as nn
from esm.tokenization import InterProQuantizedTokenizer
from esm.utils.types import FunctionAnnotation
def get_keywords_from_interpro(
    interpro_annotations,
    interpro2keywords=InterProQuantizedTokenizer().interpro2keywords,
):
    keyword_annotations_list = []
    for interpro_annotation in interpro_annotations:
        keywords = interpro2keywords.get(interpro_annotation.label, [])
        keyword_annotations_list.extend([
            FunctionAnnotation(
                label=keyword,
                start=interpro_annotation.start,
                end=interpro_annotation.end,
            )
            for keyword in keywords
        ])
    return keyword_annotations_list

In [ ]:
#protein that only has one function?
#longer sequences of all As, all Gs, or completely random
def get_label_embedding(interpro_label,sequence):
  hostProtein = ESMProtein(sequence=sequence)
  embedding_function = model.encoder.function_embed
  hostProtein.function_annotations = get_keywords_from_interpro([FunctionAnnotation(label=interpro_label, start=1, end=len(sequence))])
  hostProtein_tensor = model.encode(hostProtein)
  device = hostProtein_tensor.function.device  # Get the device of protein2_tensor.function
  embedding_function = embedding_function.to(device)  # Move embedding_function to the device

  function_embed = torch.cat(
      [
          embed_fn(funcs.to(device)) # Ensure funcs is on the same device
          for embed_fn, funcs in zip(
              embedding_function, hostProtein_tensor.function.unbind(-1)
          )
      ],
      -1,
  )

  if function_embed.shape[0] >= 3:
      row_sum = function_embed.sum(dim=0)  # Sum all rows
      row_avg = row_sum / (function_embed.shape[0] - 2)  # Divide by (number of rows - 2)
      row_avg_np = row_avg.cpu().detach().type(torch.float32).numpy()
      return row_avg_np
  else:
      return None

In [ ]:
import numpy as np
embedding_target_dict = {}  # Store the average embedding_target for each set of InterPro_ids_target

for entry, interpro_ids in embeddings_dict.items():
    # Check if the embedding_target for this set has already been calculated
    if tuple(interpro_ids['InterPro_ids_target']) not in embedding_target_dict:
        embeddings_target = []
        for interpro_id in interpro_ids['InterPro_ids_target']:
            embeddings_target.append(get_label_embedding(interpro_id, "A"))
        avg_embedding_target = np.mean(embeddings_target, axis=0)
        embedding_target_dict[tuple(interpro_ids['InterPro_ids_target'])] = avg_embedding_target
    else:
        avg_embedding_target = embedding_target_dict[tuple(interpro_ids['InterPro_ids_target'])]

    embeddings_dict[entry]['embedding_target'] = avg_embedding_target

    # Calculate embedding_original as before
    embeddings_original = []
    for interpro_id in interpro_ids['InterPro_ids_original']:
        embeddings_original.append(get_label_embedding(interpro_id, "A"))
    avg_embedding_original = np.mean(embeddings_original, axis=0)
    embeddings_dict[entry]['embedding_original'] = avg_embedding_original

In [ ]:
first_key, first_value = next(iter(embeddings_dict.items()))
print(first_key, first_value)

5R1U_1 {'InterPro_ids_target': ['IPR000980'], 'InterPro_ids_original': ['IPR034163', 'IPR021109', 'IPR001461', 'IPR001969', 'IPR033121'], 'embedding_target': array([ 0.703125  ,  0.19335938, -0.0625    , ...,  0.25390625,
       -0.37304688, -0.546875  ], dtype=float32), 'embedding_original': array([ 0.09257813,  0.0796875 , -0.11015625, ..., -0.03886719,
       -0.20625   ,  0.0328125 ], dtype=float32)}


In [ ]:
def embedding_masking_model(
    prompt,
    model,
    df,
    embeddings_dict,
    percentage=10,
):
    """
    Helper function to process a protein sequence, calculate similarities,
    and return indices for masking.

    Args:
        prompt (str): The protein sequence to be processed.
        model: The model used for protein generation and embeddings.
        df (pd.DataFrame): DataFrame containing protein data.
        embeddings_dict (dict): Dictionary storing embeddings and other details.

    Returns:
        List[int]: Indices used for masking in the sequence.
    """
    # Create an ESMProtein object
    protein = ESMProtein(sequence=prompt)

    # Configure the model for generation
    generation_config = GenerationConfig(track="function", num_steps=8)

    # Generate the protein
    generated_protein = model.generate(protein, generation_config)

    # Check if function annotations are available
    entry = df.loc[df['Sequence'] == prompt, 'Protein Accession'].iloc[0]
    if generated_protein.function_annotations is None:
        embeddings_dict[entry]['hamming_distance'] = None
        return []

    # Getting embedding for the protein
    protein_tensor = model.encode(generated_protein)
    embedding_function = model.encoder.function_embed
    device = protein_tensor.function.device  # Get the device of protein_tensor.function
    embedding_function = embedding_function.to(device)  # Move embedding_function to the device

    function_embed = torch.cat(
        [
            embed_fn(funcs.to(device))  # Ensure funcs is on the same device
            for embed_fn, funcs in zip(
                embedding_function, protein_tensor.function.unbind(-1)
            )
        ],
        -1,
    )

    # Exclude start and end tokens
    function_embed = function_embed[1:-1, :]

    # Convert the protein_tensor.function to a NumPy array
    protein_np = function_embed.cpu().detach().type(torch.float32).numpy()

    # Retrieve embedding for target function's InterPro IDs (target function = function we're trying to add in addition to, not in place of, original)
    embedding_target = embeddings_dict[entry]['embedding_target']

    # Retrieve embedding for original function's InterPro IDs
    embedding_original = embeddings_dict[entry]['embedding_original']

    # Calculate cosine similarity
    similarities_target = cosine_similarity(protein_np, embedding_target.reshape(1, -1))
    similarities_original = cosine_similarity(protein_np, embedding_original.reshape(1, -1))

    # Get ranks for similarity from highest similarity to lowest similarity for original and lowest to highest for target. This allows you to find indices with least similarity to original and most similarity to target to edit
    ranks_target = np.argsort(similarities_target.flatten())
    ranks_original = np.argsort(similarities_original.flatten())[::-1]

    #Get number of indices to mask
    num_indices = int(len(prompt) * percentage / 100)

    # Ensure we select at least 1 index
    num_indices = max(1, num_indices)

    # Sum ranks and find top 10 indices with lowest (largest) rank
    ranks = ranks_target + ranks_original
    num_negated = -1*num_indices
    indices = np.argsort(ranks)[num_negated:]

    # Store the indices in the embeddings_dict
    embeddings_dict[entry]['indices'] = indices.tolist()

    return indices.tolist()

In [ ]:
def get_random_indices(prompt, percentage):
    """
    Randomly select indices to mask based on the percentage of the prompt's length.
    """
    num_indices = int(len(prompt) * percentage / 100)
    # Ensure we select at least one index
    num_indices = max(1, num_indices)

    # Randomly select unique indices to mask
    return random.sample(range(len(prompt)), num_indices)

In [ ]:
for idx, prompt in enumerate(df['Sequence']):
  try:
    random_indices = get_random_indices(prompt, 8)
    # Replace locations in prompt corresponding to indices in 'indices' with "_"
    target_seq = df.loc[df['Sequence'] == prompt, 'Sequence'].iloc[0]
    entry = df.loc[df['Sequence'] == prompt, 'Protein Accession'].iloc[0]
    if entry not in embeddings_dict:
        continue
    for method in ['embedding', 'random']:
      if method == 'embedding':
        ##For SH2 domain, selecting 100 indices to edit
        total_length = len(prompt)  # Get the length of the list

        ################
        num_indices = min(num_residues_to_mask, total_length)  # Select 100 or the total length if it's less than 100
        ################

        percentage = (num_indices / total_length) * 100


        indices = embedding_masking_model(prompt, model, df, embeddings_dict,percentage=percentage)
        if not indices:
          continue
      elif method == 'random':
        continue
        # indices = get_random_indices(prompt, 8)
      modified_prompt = list(prompt)
      for index in indices:
          modified_prompt[index] = "_"
      modified_prompt = "".join(modified_prompt)
      protein_prompt = ESMProtein(sequence=modified_prompt)
      sequence_generation = model.generate(
          protein_prompt,
          GenerationConfig(
              track="sequence",
              num_steps=protein_prompt.sequence.count("_") // 2,
              temperature=0.5,
          ),
      )
      # print("Generated Sequence: " + str(sequence_generation.sequence))
      generated_sequence = sequence_generation.sequence
      # print("Target Sequence: " + str(target_seq))
      # Ensure sequences are of equal length
      if len(generated_sequence) != len(target_seq):
          raise ValueError("Sequences must be of the same length to calculate Hamming distance.")

      # Calculate Hamming distance
      hamming_distance = sum(1 for gen, target in zip(generated_sequence, target_seq) if gen != target)

      # Print results
      print(f'Index {idx}: {method} Hamming Distance:', hamming_distance)
      embeddings_dict[entry][f'{method}_hamming_distance'] = hamming_distance
      embeddings_dict[entry][f'{method}_edited_sequence'] = generated_sequence
      torch.cuda.empty_cache()
  except Exception as e:
    print(f"An error occurred for index {idx}: {e}")
    continue

100%|██████████| 8/8 [00:04<00:00,  1.67it/s]
/usr/local/lib/python3.11/dist-packages/esm/pretrained.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.

An error occurred for index 0: CUDA out of memory. Tried to allocate 422.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 76.12 MiB is free. Process 4711 has 14.66 GiB memory in use. Of the allocated memory 13.50 GiB is allocated by PyTorch, and 1.04 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


100%|██████████| 8/8 [00:07<00:00,  1.13it/s]


An error occurred for index 3: CUDA out of memory. Tried to allocate 150.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 24.12 MiB is free. Process 4711 has 14.71 GiB memory in use. Of the allocated memory 13.68 GiB is allocated by PyTorch, and 931.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


100%|██████████| 49/49 [00:12<00:00,  4.07it/s]


...

An error occurred for index 62: CUDA out of memory. Tried to allocate 276.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 222.12 MiB is free. Process 4711 has 14.52 GiB memory in use. Of the allocated memory 13.52 GiB is allocated by PyTorch, and 898.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


100%|██████████| 8/8 [00:02<00:00,  3.05it/s]


An error occurred for index 63: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 4711 has 14.74 GiB memory in use. Of the allocated memory 12.87 GiB is allocated by PyTorch, and 1.74 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


100%|██████████| 50/50 [00:12<00:00,  4.15it/s]

Index 64: embedding Hamming Distance: 41


In [ ]:
# prompt: go through embeddings_dict and turn it into a dataframe where the key becomes the first column and each key value pair in the value dict becomes subsequent columns

df_embeddings = pd.DataFrame.from_dict(embeddings_dict, orient='index')

# Reset the index to make the keys a regular column
df_embeddings = df_embeddings.reset_index()

# Rename the 'index' column to 'Protein Accession'
df_embeddings = df_embeddings.rename(columns={'index': 'Protein Accession'})

df_embeddings = update_dataframe_with_sequences(df_embeddings, fasta_dict)
df_embeddings


,Protein Accession,InterPro_ids_target,InterPro_ids_original,embedding_target,embedding_original,hamming_distance,indices,embedding_hamming_distance,embedding_edited_sequence,Sequence
0,5R1U_1,[IPR000980],"[IPR034163, IPR021109, IPR001461, IPR001969, I...","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[0.09257813, 0.0796875, -0.11015625, -0.067968...",NaN,NaN,NaN,NaN,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
1,6KFD_1,[IPR000980],[IPR035992],"[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",NaN,NaN,NaN,NaN,LTCDQLPKAAINPIQEFIDSNPLEFEYVLTETFECTTRIYVQPARW...
2,6L9T_1,[IPR000980],"[IPR019791, IPR037120, IPR010255]","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[-0.17578125, 0.07470703, 0.54296875, -0.41210...",NaN,NaN,NaN,NaN,SWEVGCGAPVPLVKCDENSPYRTITGDCNNRRSPALGAANRALARW...
3,6LM2_1,[IPR000980],"[IPR012674, IPR015304]","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[-0.54296875, 0.58203125, 0.055908203, -0.1972...",NaN,"[63, 140, 128, 107, 137, 125, 164, 154, 160, 1...",89.0,MGTAVEQKIANGVFDDANVQNRTLSDWDGIWQSKGGFVINSGLDKH...,ALTEVEQKAANGVFDDANVQNRTLSDWDGVWQSVYPLLQSGKLDPV...
4,6LOV_1,[IPR000980],"[IPR016139, IPR036041, IPR017989, IPR017988, I...","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[-0.049804688, 0.16145833, 0.140625, -0.052083...",NaN,NaN,NaN,NaN,MSRFSVLSFLILAIFLGGSIVKGDVSFRLSGADPRSYGMFIKDLRN...
...,...,...,...,...,...,...,...,...,...,...
59,8WDG_1,[IPR000980],"[IPR001998, IPR036237, IPR013022, IPR013453]","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[-0.32910156, 0.33068848, 0.024414062, -0.2451...",NaN,NaN,NaN,NaN,YQPTPEDRFTFGLWTVGWQGRDPFGDATRRALDPVESVRRLAELGA...
60,8YJJ_1,[IPR000980],"[IPR001137, IPR013319, IPR013320, IPR018208, I...","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[-0.52246094, -0.014811198, 0.103515625, 0.142...",NaN,"[87, 60, 59, 58, 57, 56, 55, 172, 173, 174, 9,...",38.0,MVIGNQTGTDNGYFYSYWTDGHGNVTYTNGPGGYSVSFGSGSGNFV...,QTIQPGTGYNNGYFYSYWNDGHGGVTYTNGPGGQFSVNWSNSGNFV...
61,9C6E_1,[IPR000980],"[IPR014756, IPR000698, IPR011022, IPR017864, I...","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[-0.5924944, -0.005998884, 0.091796875, 0.4111...",NaN,NaN,NaN,NaN,ANKPAPNHVIFKKISRDKSVTIYLGKRDYIDHVERVEPVDGVVLVD...
62,9EAT_1,[IPR000980],"[IPR015892, IPR001765, IPR036874]","[0.703125, 0.19335938, -0.0625, -0.05078125, -...","[0.05419922, 0.26139322, 0.057210285, -0.02864...",NaN,NaN,NaN,NaN,MKDIDTLISNNALWSKMLVEEDPGFFEKLAQAQKPRFLWIGCSDSR...


In [ ]:
df_embeddings.to_csv(f'interpro_scan_test_data_w_edited_sequences_SH2_{num_residues_to_mask}_residues.csv', index = False)

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Extract Hamming distances for 'embedding' and 'random' methods
# embedding_distances = []
# random_distances = []

# # Iterate over embeddings_dict to gather the hamming distances
# for key, entry in embeddings_dict.items():
#     embedding_distance = entry.get('embedding_hamming_distance', None)
#     random_distance = entry.get('random_hamming_distance', None)
#     seq_length = len(df.loc[df['Protein Accession'] == key, 'Sequence'].iloc[0])
#     embeddings_dict[key]['seq_length'] = seq_length

#     # Skip None values and collect valid distances (only compare on proteins w/ both values)
#     if embedding_distance is not None and random_distance is not None:
#         embedding_distances.append(embedding_distance/seq_length)
#         random_distances.append(random_distance/seq_length)

# # Calculate average and standard deviation for both methods
# embedding_avg = np.mean(embedding_distances) if embedding_distances else 0
# embedding_std = np.std(embedding_distances) if embedding_distances else 0
# print(embedding_avg)
# print(embedding_std)

# random_avg = np.mean(random_distances) if random_distances else 0
# random_std = np.std(random_distances) if random_distances else 0
# print(random_avg)
# print(random_std)

# # Prepare data for the bar plot
# methods = ['Embedding', 'Random']
# averages = [embedding_avg, random_avg]
# std_devs = [embedding_std, random_std]

# # Create the bar plot
# fig, ax = plt.subplots()

# # Set positions for the bars
# bar_width = 0.35
# index = np.arange(len(methods))

# # Plot the bars with error bars
# ax.bar(index, averages, bar_width, yerr=std_devs, capsize=5, label='Average Hamming Distance')

# # Customize the plot
# ax.set_xlabel('Method')
# ax.set_ylabel('Hamming Distance')
# ax.set_title('Average Hamming Distance and Standard Deviation by Method')
# ax.set_xticks(index)
# ax.set_xticklabels(methods)
# ax.legend()

# # Display the plot
# plt.show()

In [ ]:
# import json
# import numpy as np

# # Specify the file path where you want to save the JSON file
# file_path = 'hamming_distance_dict.json'

# # Define a custom encoder to handle NumPy arrays
# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()  # Convert NumPy array to list
#         return json.JSONEncoder.default(self, obj)

# # Save the dictionary to a JSON file using the custom encoder
# with open(file_path, 'w') as json_file:
#     json.dump(embeddings_dict, json_file, indent=4, cls=NumpyEncoder)
#     # Use cls=NumpyEncoder to apply the custom encoding